In [1]:
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from functools import reduce
from pylab import rcParams
rcParams['figure.figsize'] = 10,10

In [2]:
# benchmarking throughput

target_ids = [
    '30I8399019000',
    '30I8399019001',
    '30I8399019002',
    '30I8399019003',
    '30I8399019004',
    '30I8399019005',
    '30I8399019006',
    '30I8399019007',
    '30I8399019008',
    '30I8399019009',
    '30I8399019010',
    '30I8399019011',
    '30I8399019012',
    '30I8399019013',
]
base_dir = '../'
download_dir = 'tmp/collect-all'
benchmark_dir = 'tmp/benchmark_throughput'

def parse_elapsed(lines):
    ret = {}
    for x in lines:
        key, value = x.split('\t')
        m, s = value.replace('s', '').split('m')
        ret[key] = float(m)*60 + float(s)
    return ret

def file_stat(files):
    ret = {}
    ret['file_num'] = len(files)
    ret['file_size'] = reduce(lambda a,b: a+b ,map(os.path.getsize, files))

    return ret

cmd_rm_stat = 'cd {base_dir} && mkdir -p {benchmark_dir} && rm -f {benchmark_dir}/stat-{id}.json'
cmd_calc_stat = 'cd {base_dir} && time python3 lastomesh.py --local-scheduler --workers 4 DownloadShizuokaPCD --product-id {id} --output-dir {benchmark_dir} --work-dir {download_dir}/{id}'

throughputs = []
for id in target_ids:

    # ファイル数, 容量
    las_files = list(glob.glob(os.path.join(base_dir, download_dir, id, '*.las')))
    fstat = file_stat(las_files)
    !{cmd_rm_stat.format(base_dir=base_dir, benchmark_dir=benchmark_dir, id=id)}

    # 読み込み&特徴量算出 時間計測
    output = !{cmd_calc_stat.format(base_dir=base_dir, download_dir=download_dir, benchmark_dir=benchmark_dir, id=id)}
    elapsed = parse_elapsed(output[-3:])

    # 点の数
    filepath = os.path.join(base_dir, benchmark_dir, 'stat-{}.json'.format(id))
    with open(filepath, 'r') as f:
        pcd_stat = json.load(f)

    # 結果まとめる
    record = dict(fstat, **elapsed, point_num=pcd_stat['shape']['value'][0], id=id)
    print(record)
    throughputs.append(record)

df_throughputs = pd.DataFrame(throughputs)
df_throughputs

{'file_num': 4, 'file_size': 641946890, 'real': 10.039, 'user': 15.714, 'sys': 3.699, 'point_num': 18880735, 'id': '30I8399019000'}
{'file_num': 4, 'file_size': 635859292, 'real': 9.775, 'user': 15.569, 'sys': 3.523, 'point_num': 18701690, 'id': '30I8399019001'}
{'file_num': 4, 'file_size': 641868078, 'real': 9.694, 'user': 15.58, 'sys': 3.521, 'point_num': 18878420, 'id': '30I8399019002'}
{'file_num': 4, 'file_size': 649677538, 'real': 9.907, 'user': 15.903, 'sys': 3.527, 'point_num': 19108110, 'id': '30I8399019003'}
{'file_num': 4, 'file_size': 648882006, 'real': 9.976, 'user': 15.691, 'sys': 3.608, 'point_num': 19084710, 'id': '30I8399019004'}
{'file_num': 4, 'file_size': 637587546, 'real': 9.597, 'user': 15.549, 'sys': 3.478, 'point_num': 18752520, 'id': '30I8399019005'}
{'file_num': 3, 'file_size': 605604597, 'real': 9.191, 'user': 14.444, 'sys': 3.618, 'point_num': 17811860, 'id': '30I8399019006'}
{'file_num': 3, 'file_size': 618411615, 'real': 9.513, 'user': 15.277, 'sys': 3.464

,file_num,file_size,real,user,sys,point_num,id
0,4,641946890,10.039,15.714,3.699,18880735,30I8399019000
1,4,635859292,9.775,15.569,3.523,18701690,30I8399019001
2,4,641868078,9.694,15.580,3.521,18878420,30I8399019002
3,4,649677538,9.907,15.903,3.527,19108110,30I8399019003
4,4,648882006,9.976,15.691,3.608,19084710,30I8399019004
5,4,637587546,9.597,15.549,3.478,18752520,30I8399019005
6,3,605604597,9.191,14.444,3.618,17811860,30I8399019006
7,3,618411615,9.513,15.277,3.464,18188535,30I8399019007
8,4,629775264,9.663,15.356,3.478,18522750,30I8399019008
9,4,650957842,10.032,15.756,3.737,19145765,30I8399019009


In [3]:
throughput_point = df_throughputs['point_num'].sum() / df_throughputs['real'].sum()
file_size_sum = df_throughputs['file_size'].sum() / (1024 ** 3)
elapsed_per_1gb = df_throughputs['real'].sum() / file_size_sum
print(throughput_point, elapsed_per_1gb, file_size_sum, df_throughputs['point_num'].sum())

1910779.2554727658 16.527578929418056 8.380174772813916 264650570


In [4]:
# benchmarking downsampling

target_projects = {
    'mms':[
        '30I8399019000',
        '30I8399019001',
        '30I8399019002',
        '30I8399019003',
        '30I8399019004',
        '30I8399019005',
        '30I8399019006',
        '30I8399019007',
        '30I8399019008',
        '30I8399019009',
    ],
    'construction': [
        '28XXX00040001',
        '29D6152011105',
        '29K2033011103',
        '29K2461011102',
        '29K3481011101',
        '29W9350011101',
        '30D0230011102',
        '30D0721011102',
        '30D3703011102',
        '30D7318011101',
    ],
    'building': [
        '01R0107011318',
        '01R0107021318',
        '01R0107031318',
        '01R0107041318',
        '01R0107051318',
        '01R0107061318',
        '01R0107071318',
        '29XXX00010002',
        '30XXX03010001',
        '31XXX07010001',
    ],
    'terrain': [
        '28XXX00030001',
        '28XXX00030002',
        '28XXX00030003',
        '28XXX00030004',
        '30XXX00010001',
        '30XXX00010002',
        '30XXX00010003',
        '30XXX00010004',
        '30XXX00010005',
        '30XXX00010062',
    ],
}
base_dir = '../'
download_dir = 'tmp/collect-all'
benchmark_dir = 'tmp/benchmark_throughput'

cmd_rm_ply = 'cd {base_dir} && mkdir -p {benchmark_dir} && rm -f {benchmark_dir}/pcd-{id}.ply'
cmd_calc_ply = 'cd {base_dir} && time python3 lastomesh.py --local-scheduler --workers 4 CreateMeshFromLasData --product-id {id} --output-dir {benchmark_dir} --work-dir {download_dir}/{id} --output-filename pcd-{id}.ply --skip-meshing true'

downsampling = []
for group, target_ids in target_projects.items():
    for id in target_ids:
        print(id)
        # 読み込み&特徴量算出 時間計測
        !{cmd_rm_ply.format(base_dir=base_dir, benchmark_dir=benchmark_dir, id=id)}
        output = !{cmd_calc_ply.format(base_dir=base_dir, download_dir=download_dir, benchmark_dir=benchmark_dir, id=id)}
        elapsed = parse_elapsed(output[-3:])

        # ファイル数, 容量
        las_files = list(glob.glob(os.path.join(base_dir, download_dir, id, '*.las')))
        fstat = file_stat(las_files)

        ply_files = list(glob.glob(os.path.join(base_dir, benchmark_dir, 'pcd-{}.ply'.format(id))))
        fstat_ply = file_stat(ply_files)

        # 結果まとめる
        record = dict(ply_size=fstat_ply['file_size'], **fstat, **elapsed, id=id, group=group)
        # print(record)
        downsampling.append(record)

df_downsampling = pd.DataFrame(downsampling)
df_downsampling

30I8399019000
30I8399019001
30I8399019002
30I8399019003
30I8399019004
30I8399019005
30I8399019006
30I8399019007
30I8399019008
30I8399019009
28XXX00040001
29D6152011105
29K2033011103
29K2461011102
29K3481011101
29W9350011101
30D0230011102
30D0721011102
30D3703011102
30D7318011101
01R0107011318
01R0107021318
01R0107031318
01R0107041318
01R0107051318
01R0107061318
01R0107071318
29XXX00010002
30XXX03010001
31XXX07010001
28XXX00030001
28XXX00030002
28XXX00030003
28XXX00030004
30XXX00010001
30XXX00010002
30XXX00010003
30XXX00010004
30XXX00010005
30XXX00010062


,ply_size,file_num,file_size,real,user,sys,id,group
0,73619652,4,641946890,6.002,5.458,1.474,30I8399019000,mms
1,75579447,4,635859292,6.026,5.483,1.447,30I8399019001,mms
2,75647244,4,641868078,6.074,5.567,1.434,30I8399019002,mms
3,77465991,4,649677538,6.196,5.651,1.477,30I8399019003,mms
4,73542675,4,648882006,6.093,5.500,1.523,30I8399019004,mms
5,75265761,4,637587546,6.088,5.529,1.470,30I8399019005,mms
6,75237492,3,605604597,6.024,5.516,1.433,30I8399019006,mms
7,74964036,3,618411615,5.958,5.436,1.376,30I8399019007,mms
8,75399951,4,629775264,6.023,5.439,1.510,30I8399019008,mms
9,77269512,4,650957842,6.191,5.565,1.536,30I8399019009,mms


In [5]:
downsampling_mean = df_downsampling.mean()
downsampling_sum = df_downsampling.sum()

reduce_ratio = downsampling_sum['ply_size'] / downsampling_sum['file_size']
elapsed_per_1gb = downsampling_sum['real'] / (downsampling_sum['file_size'] / 1024**3 )
print(elapsed_per_1gb, reduce_ratio, downsampling_sum['ply_size'] / 1024**2 / 40)

17.94099955679196 0.12323122806648364 73.02768614292145


In [6]:
df_downsampling['file_size_mb'] = df_downsampling['file_size'] / 1024**2
df_downsampling['ply_size_mb'] = df_downsampling['ply_size'] / 1024**2

for key, downsampling_sum in df_downsampling.groupby('group').sum().iterrows():
    reduce_ratio = downsampling_sum['ply_size'] / downsampling_sum['file_size']
    elapsed_per_1gb = downsampling_sum['real'] / (downsampling_sum['file_size'] / 1024**3 )
    print(key, elapsed_per_1gb, reduce_ratio)

df_downsampling.groupby('group').mean()

building 20.591203971344815 0.12402059640716431
construction 23.390010219932332 0.12596893281667568
mms 10.242679245584949 0.11854152722385884
terrain 17.093709609621694 0.12464337712348633


,ply_size,file_num,file_size,real,user,sys,file_size_mb,ply_size_mb
group,,,,,,,,
building,90308370.3,4.0,728172359.4,13.9642,9.5333,2.2642,694.439277,86.124773
construction,78364086.1,3.6,622090577.0,13.5514,10.2362,2.3914,593.271806,74.733816
mms,75399176.1,3.8,636057066.8,6.0675,5.5144,1.4680,606.591288,71.906258
terrain,62228683.6,2.9,499253831.5,7.9480,4.9996,1.7755,476.125556,59.345897


In [7]:
base_dir = '../'
download_dir = 'tmp/collect-all'
benchmark_dir = 'tmp/benchmark_throughput'

cmd_rm_ply = 'cd {base_dir} && mkdir -p {benchmark_dir} && rm -f {benchmark_dir}/mesh-{id}.ply'
cmd_calc_ply = 'cd {base_dir} && time python3 lastomesh.py --local-scheduler --workers 4 CreateMeshFromLasData --product-id {id} --output-dir {benchmark_dir} --work-dir {download_dir}/{id} --output-filename mesh-{id}.ply --simplify-type vertex-clustering'

meshing = []
for group, target_ids in target_projects.items():
    for id in target_ids:
        print(id)
        # 読み込み&特徴量算出 時間計測
        !{cmd_rm_ply.format(base_dir=base_dir, benchmark_dir=benchmark_dir, id=id)}
        output = !{cmd_calc_ply.format(base_dir=base_dir, download_dir=download_dir, benchmark_dir=benchmark_dir, id=id)}
        elapsed = parse_elapsed(output[-3:])

        # ファイル数, 容量
        las_files = list(glob.glob(os.path.join(base_dir, download_dir, id, '*.las')))
        fstat = file_stat(las_files)

        ply_files = list(glob.glob(os.path.join(base_dir, benchmark_dir, 'mesh-{}.ply'.format(id))))
        fstat_ply = file_stat(ply_files)

        # 結果まとめる
        record = dict(ply_size=fstat_ply['file_size'], **fstat, **elapsed, id=id, group=group)
        # print(record)
        meshing.append(record)

df_meshing = pd.DataFrame(meshing)
df_meshing

30I8399019000
30I8399019001
30I8399019002
30I8399019003
30I8399019004
30I8399019005
30I8399019006
30I8399019007
30I8399019008
30I8399019009
28XXX00040001
29D6152011105
29K2033011103
29K3481011101
29W9350011101
30D0230011102
30D0721011102
30D3703011102
30D7318011101
30K2664011102
31K2650011102
01R0107011318
01R0107021318
01R0107031318
01R0107041318
01R0107051318
01R0107061318
01R0107071318
29XXX00010002
30XXX03010001
31XXX07010001
28XXX00030001
28XXX00030002
28XXX00030003
28XXX00030004
30XXX00010001
30XXX00010002
30XXX00010003
30XXX00010004
30XXX00010005
30XXX00010062


,ply_size,file_num,file_size,real,user,sys,id,group
0,10981199,4,641946890,38.633,210.969,10.619,30I8399019000,mms
1,10606062,4,635859292,30.777,155.412,8.399,30I8399019001,mms
2,10048693,4,641868078,22.377,93.551,6.369,30I8399019002,mms
3,9963153,4,649677538,24.105,107.598,6.521,30I8399019003,mms
4,10368664,4,648882006,25.248,115.607,6.717,30I8399019004,mms
5,10217828,4,637587546,22.994,98.924,6.038,30I8399019005,mms
6,8947008,3,605604597,22.548,98.743,6.240,30I8399019006,mms
7,9325417,3,618411615,26.818,128.235,7.696,30I8399019007,mms
8,9164972,4,629775264,25.999,122.753,7.832,30I8399019008,mms
9,9690015,4,650957842,24.103,107.275,6.695,30I8399019009,mms


In [8]:
meshing_mean = df_meshing.mean()
meshing_sum = df_meshing.sum()

reduce_ratio = meshing_sum['ply_size'] / meshing_sum['file_size']
elapsed_per_1gb = meshing_sum['real'] / (meshing_sum['file_size'] / 1024**3 )
print(elapsed_per_1gb, reduce_ratio, meshing_sum['file_size'] / 1024**2 / 40, meshing_sum['ply_size'] / 1024**2 / 40)
meshing_sum

64.45968515485137 0.016578674243309163 583.5605294942856 9.674659919738769


ply_size                                             405784648
file_num                                                   143
file_size                                          24476302631
real                                                   1469.38
user                                                   7104.51
sys                                                    381.814
id           30I839901900030I839901900130I839901900230I8399...
group        mmsmmsmmsmmsmmsmmsmmsmmsmmsmmsconstructioncons...
dtype: object

In [9]:
df_meshing['file_size_mb'] = df_meshing['file_size'] / 1024**2
df_meshing['ply_size_mb'] = df_meshing['ply_size'] / 1024**2

for key, meshing_sum in df_meshing.groupby('group').sum().iterrows():
    reduce_ratio = meshing_sum['ply_size'] / meshing_sum['file_size']
    elapsed_per_1gb = meshing_sum['real'] / (meshing_sum['file_size'] / 1024**3 )
    print(key, elapsed_per_1gb, reduce_ratio)

df_meshing.groupby('group').mean()

building 70.19562402230672 0.01806078839745644
construction 50.06770839239708 0.016481949425397158
mms 44.49922924589507 0.015613852307253385
terrain 98.36281410164722 0.0157593490596977


,ply_size,file_num,file_size,real,user,sys,file_size_mb,ply_size_mb
group,,,,,,,,
building,1.315137e+07,4.000000,7.281724e+08,47.604100,228.930000,10.6140,694.439277,12.542121
construction,8.752619e+06,3.272727,5.310427e+08,24.762091,94.526909,5.9970,506.441815,8.347148
mms,9.931301e+06,3.800000,6.360571e+08,26.360200,123.906700,7.3126,606.591288,9.471227
terrain,7.867915e+06,2.900000,4.992538e+08,45.735400,253.634800,13.6581,476.125556,7.503429
